# Introduction to APIs 

## Setup

Complete the "Getting started" here: https://developer.spotify.com/documentation/web-api

### Collecting Secrets
Rather than keeping secrets where they might be accidentally shared, we will call them from a special file outside of our GitHub repos. 

In [ ]:
import pandas as pd
import os

secrets_location = "../../../../Secrets/secrets_for_github.xlsx"
secrets_db = pd.read_excel(secrets_location)

os.environ['SPOTIPY_CLIENT_ID'] = secrets_db.loc[secrets_db["code_name"] == "spotify_cid"]["code"].values.item()
os.environ['SPOTIPY_CLIENT_SECRET'] = secrets_db.loc[secrets_db["code_name"] == "spotify_secret"]["code"].values.item()
os.environ['SPOTIPY_REDIRECT_URI'] = "http://localhost:8888/callback"

### Authorize
Follow the instructions here: https://spotipy.readthedocs.io/en/2.24.0/#authorization-code-flow

Be sure to install spotipy via Command Prompt/Terminal with 
> pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

In [ ]:
username = ""
client_credentials_manager = SpotifyClientCredentials() 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = "user-top-read"
token = util.prompt_for_user_token(username, scope)

In [ ]:
def extract_artist_name(row):
    full_artist_info = row["artists"]
    artist_list = [full_artist_info[index]["name"] for index in range(0, len(full_artist_info))]
    return artist_list

In [ ]:
sp = spotipy.Spotify(auth=token)
results = sp.current_user_top_tracks(limit=50,offset=0,time_range='long_term')

full_items_df = pd.DataFrame([item for item in results["items"]])
items_df = pd.DataFrame([item for item in results["items"]])[["artists", "name", "popularity"]]
items_df["artist_name"] = items_df.apply(extract_artist_name, axis=1)
items_df = items_df.explode("artist_name")
items_df = items_df[["artist_name", "name", "popularity"]]

In [ ]:
items_df

In [ ]:
full_items_df.keys()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
def encode_artist_names(row):
    return row["artist_name"].encode("ascii",errors="ignore")

In [ ]:
more_than_1_series = items_df["artist_name"].value_counts()
top_artists_series = more_than_1_series[more_than_1_series > 1]
top_artists_df = pd.DataFrame({"artist_name": top_artists_series.index, "song_count": top_artists_series.values})
top_artists_df["artist_name"] = top_artists_df.apply(encode_artist_names, axis=1)
print(top_artists_df)

In [ ]:
ax = sb.histplot(data=top_artists_df, weights="song_count", y="artist_name")

# Forces axes to be unlabled
ax.set_ylabel('')    
ax.set_xlabel('')

ax.set_title('Songs per Artist in Top 50', fontsize=16, fontweight='heavy')

plt.savefig('top50_songs_per_artist.jpg', bbox_inches="tight")